In [1]:
from src.config.data_loader_config import DATA_LOADER_CONFIG, OPTIMIZER_CONFIG
from src.data_loader.loader import Dataloader
from src.model.model import Model, Models, LossFunctions
from src.trainer.predict import save_result
import src.callback as callback
import pytorch_lightning as pl

# Parameters 설정
batch_size = DATA_LOADER_CONFIG['batch_size']
shuffle = DATA_LOADER_CONFIG['shuffle']
learning_rate = OPTIMIZER_CONFIG['learning_rate']
max_epoch = OPTIMIZER_CONFIG['max_epoch']

In [ ]:
# 모델 선언
# select Model -> roberta_base, roberta_small, roberta_large, electra_base, electra_base_v3
model_name = 'electra_base_v3'
model = Model(Models.electra_base_v3, learning_rate, LossFunctions.hu_loss)

# Dataloader 선언
dataloader = Dataloader(batch_size, shuffle, Models.electra_base_v3, check_aug=True)

# callback 정의
epoch_print_callback = callback.EpochPrintCallback()
checkpoint_callback = callback.ModelCheckpoint(model_name=model_name)
early_stopping = callback.EarlyStopping()
lr_monitor = callback.LearningRateMonitor()

# 학습

In [ ]:
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=max_epoch, callbacks=[lr_monitor, epoch_print_callback,checkpoint_callback, early_stopping])
trainer.fit(model=model, datamodule=dataloader)

# 추론


In [ ]:
# 가장 좋은 모델 불러오기
best_model_path = checkpoint_callback.best_model_path
model = Model.load_from_checkpoint(best_model_path)
trainer.test(model=model, datamodule=dataloader)
# 추론
predictions = trainer.predict(model=model, datamodule=dataloader)

# 결과 저장
save_result(predictions, model_name, max_epoch)